In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib as mpl
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
from matplotlib.colors import LinearSegmentedColormap

In [ ]:
mpl.rcParams['figure.dpi']=300
palette = {'A82331', 'DC5663'}
background_color = '#FFF4EE'
main_color = '#A82331'

In [ ]:
plt.rcParams['axes.facecolor'] = 'white'

In [ ]:
boroughs = gpd.read_file('../data/Borough Boundaries.geojson')
boroughs.head(20)

In [ ]:
boroughs.plot()

In [ ]:
annual_report = pd.read_csv('../data/DOHMH_HIV_AIDS_Annual_Report_20240129.csv')
borough_annual_report = annual_report[(annual_report['Borough'] != 'All') & \
                                  (annual_report['Year'] == 2021) & \
                                  (annual_report['Race'] == 'All') & \
                                  (annual_report['Gender'] == 'All') & \
                                  (annual_report['Age'] == 'All') & \
                                  (annual_report['UHF'] == 'All')]
borough_annual_report

In [ ]:
bor_p = pd.merge(boroughs, borough_annual_report, left_on='boro_name', right_on='Borough')
bor_p

In [ ]:
colors = [(0, '#EA99A1'), (0.5, '#DC5663'), (1, '#A82331')]
red_cmap = LinearSegmentedColormap.from_list('custom_colormap', colors)

In [ ]:
bor_p['coords'] = bor_p['geometry'].apply(lambda x: x.representative_point().coords[:])
bor_p['coords'] = [coords[0] for coords in bor_p['coords']]

In [ ]:
plt.figure(figsize=(8, 8))
ax = bor_p.plot(column = 'HIV diagnoses', legend=True, cmap=red_cmap)
for idx, row in bor_p.iterrows():
    plt.annotate(text=row['boro_name'], xy=row['coords'], horizontalalignment='center')
ax.set_xlabel(None)
ax.set_ylabel(None)
#ax.set_facecolor(background_color)
plt.gca().set_xticklabels([])
plt.gca().set_yticklabels([])
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['left'].set_visible(False)
plt.gca().spines['bottom'].set_visible(False)

In [ ]:
annual_report = pd.read_csv('../data/DOHMH_HIV_AIDS_Annual_Report_20240129.csv')
uhf_annual_report = annual_report[(annual_report['Borough'] != 'All') & \
                                  (annual_report['Year'] == 2021) & \
                                  (annual_report['Race'] == 'All') & \
                                  (annual_report['Gender'] == 'All') & \
                                  (annual_report['Age'] == 'All') & \
                                  (annual_report['UHF'] != 'All')]
uhf_annual_report.head()

In [ ]:
uhf_codes = pd.read_csv('../data/uhf_codes.csv')
uhf_codes = uhf_codes[['UHF_Code', 'Neighborhood']]
uhf_codes.head()

In [ ]:
uhf_annual_report_merged = pd.merge(left=uhf_annual_report, right=uhf_codes, left_on='UHF', right_on='Neighborhood', how='left')
uhf_annual_report_merged.head()

In [ ]:
neighborhoods = gpd.read_file('../data/uhf42.topo.json')
neighborhoods['id'] = neighborhoods['id'].astype('int64')
neighborhoods = neighborhoods.iloc[0:-1]
neighborhoods

In [ ]:
neighborhoods_hiv = pd.merge(left=uhf_annual_report_merged, right=neighborhoods, left_on='UHF_Code', right_on='id', how='left')

In [ ]:
neighborhoods_hiv = gpd.GeoDataFrame(neighborhoods_hiv)
neighborhoods_hiv.info()

In [ ]:
neighborhoods_hiv['coords'] = neighborhoods_hiv['geometry'].apply(lambda x: x.representative_point().coords[:])
neighborhoods_hiv['coords'] = [coords[0] for coords in neighborhoods_hiv['coords']]

In [ ]:
#neighborhoods_hiv.plot()
ax = neighborhoods_hiv.plot(column = 'HIV diagnoses', legend=True, cmap=red_cmap)
# for idx, row in neighborhoods_hiv.iterrows():
#     plt.annotate(text=row['UHF_Code'], xy=row['coords'], horizontalalignment='center')
ax.set_xlabel(None)
ax.set_ylabel(None)
plt.gca().set_xticklabels([])
plt.gca().set_yticklabels([])
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['left'].set_visible(False)
plt.gca().spines['bottom'].set_visible(False)

In [ ]:
# uhf_annual_report_merged2 = pd.merge(left=uhf_annual_report, right=uhf42, left_on='UHF', right_on='Neighborhood', how='left')
# uhf_annual_report_merged2.head()

In [ ]:
# mapping testing locations
testing_locations = pd.read_csv('../data/HIV_Testing_Locations_20240129.csv')
testing_locations['Borough'] = testing_locations['Borough'].str.lower().str.title()
testing_locations = testing_locations[['Borough', 'Address']].groupby(by='Borough').count().reset_index()
testing_locations= testing_locations.rename(columns={'Address': 'test_location_count'})
testing_locations.head(10)

In [ ]:
map_by_borough_testing = pd.merge(bor_p, testing_locations, how='left', left_on='boro_name', right_on='Borough')
map_by_borough_testing.head()

In [ ]:
plt.figure(figsize=(8, 8))
ax = map_by_borough_testing.plot(column = 'test_location_count', legend=True, cmap=red_cmap)
for idx, row in bor_p.iterrows():
    plt.annotate(text=row['boro_name'], xy=row['coords'], horizontalalignment='center')
ax.set_xlabel(None)
ax.set_ylabel(None)
plt.gca().set_xticklabels([])
plt.gca().set_yticklabels([])
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['left'].set_visible(False)
plt.gca().spines['bottom'].set_visible(False)

Maps for condom distribution locations.

In [ ]:
active_condom_distrib = pd.read_csv('../data/clean_active_cond_dist.csv')
active_condom_distrib.info()

In [ ]:
ac_geo = gpd.GeoDataFrame(active_condom_distrib,
                        crs = map_by_borough_testing.crs,
                        geometry=gpd.points_from_xy(active_condom_distrib.Longitude,
                            active_condom_distrib.Latitude))

In [ ]:
type(ac_geo)

In [ ]:
ac_map = gpd.sjoin(ac_geo, map_by_borough_testing, predicate = 'within')

In [ ]:
ac_map.head()

In [ ]:
ax = map_by_borough_testing.plot(figsize = (8, 10), color = 'powderblue')
ac_map.plot( ax = ax, column = 'Borough')
plt.show()